In [99]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

Import & Read Data 


In [100]:
dfTC = pd.read_csv(r'C://Users//austi//ConstructionTimeSeriesDatav2.csv')
dfTC

,Month,MonthYear,TotalConstruction,PrivateConstruction,PublicConstruction
0,1,2-Jan,59516,45273,14243
1,2,2-Feb,58588,44475,14113
2,3,2-Mar,63782,49396,14386
3,4,2-Apr,69504,53283,16221
4,5,2-May,73384,55009,18375
...,...,...,...,...,...
141,142,13-Oct,85193,59370,25823
142,143,13-Nov,77931,55191,22741
143,144,13-Dec,71502,52132,19369
144,145,14-Jan,64661,47827,16834


Find if Private Construction has a linear trend

In [101]:
from scipy import stats 

slope,intercept,r_value,p_value,std_err = stats.linregress(dfTC.index,dfTC['PrivateConstruction'])
print('intercept =',intercept,' slope =',slope,'p_value =',p_value)

intercept = 66985.60357841766  slope = -132.5985944684912 p_value = 3.965217656244861e-08


Private Construction does not have a linear trend P-value = 3.965

In [103]:
def sqErr(row):
    return (row['PrivateConstruction'] - row['MovAvg'])**2

dfTC['MovAvg'] = np.NaN
dfTC['sqErr'] = np.NaN

dfTC.index = pd.DatetimeIndex(freq='m', start=pd.Timestamp(year=2002, month=1, day=31), periods = len(dfTC['PrivateConstruction']))

window = 36 
window = window - window % 2

for i in range (int(window/2),dfTC.shape[0]-int(window/2)):
    dfTC.loc[dfTC.index[i],'MovAvg'] = (0.5*dfTC.iloc[i - int(window/2)]['PrivateConstruction'] + dfTC.iloc[i - int(window/2)+1:i + int(window/2)]['PrivateConstruction'].sum() + .5*dfTC.iloc[i + int(window/2)]['PrivateConstruction'])/float(window)

dfTC['sqErr'] = (dfTC['PrivateConstruction'] - dfTC['MovAvg'])**2

dfTC.dropna(how='any',inplace=True)

C:\Users\austi\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Creating a DatetimeIndex by passing range endpoints is deprecated.  Use `pandas.date_range` instead.
  import sys


In [87]:
dfTC['R1Private'] = dfTC['PrivateConstruction'] - dfTC['MovAvg']
dfTC['R1ErrorPrivate'] = abs((dfTC['PrivateConstruction'] - dfTC['R1Private'])/dfTC['PrivateConstruction'])
dfTC.style.format({
    'MovAvg': '{:.1f}'.format,
    'sqErr': '{:,.1f}'.format,
    'R1Private': '{:,.1f}'.format,
    'R1ErrorPrivate': '{:,.3f}'.format
})

,Month,MonthYear,TotalConstruction,PrivateConstruction,PublicConstruction,MovAvg,sqErr,R1Private,R1ErrorPrivate
2003-07-31 00:00:00,19,3-Jul,82971,62216,20755,57969.4,"18,033,352.0","4,246.6",0.932
2003-08-31 00:00:00,20,3-Aug,85191,63234,21957,58314.9,"24,197,107.6","4,919.1",0.922
2003-09-30 00:00:00,21,3-Sep,83841,61767,22075,58718.9,"9,290,896.7","3,048.1",0.951
2003-10-31 00:00:00,22,3-Oct,83133,62813,20321,59143.6,"13,464,822.5","3,669.4",0.942
2003-11-30 00:00:00,23,3-Nov,77915,60115,17801,59593.5,"271,918.8",521.5,0.991
2003-12-31 00:00:00,24,3-Dec,71050,55584,15466,60111.7,"20,500,394.3","-4,527.7",1.081
2004-01-31 00:00:00,25,4-Jan,64934,50975,13959,60687.3,"94,329,418.8","-9,712.3",1.191
2004-02-29 00:00:00,26,4-Feb,64138,50449,13689,61335.4,"118,514,067.8","-10,886.4",1.216
2004-03-31 00:00:00,27,4-Mar,73238,57796,15443,62017.4,"17,820,241.4","-4,221.4",1.073
2004-04-30 00:00:00,28,4-Apr,78354,61177,17177,62701.1,"2,322,787.7","-1,524.1",1.025


In [88]:
maxCorr = 0.0
period = np.NaN
for i in range(1,37):
    corr = dfTC['R1Private'].autocorr(lag=i)
    print('Correlation, lag ',i,'   ',corr)
    if corr > maxCorr:
        maxCorr = corr
        period = i
print('period = ',period,'     Maximum Correlation = ',maxCorr)

Correlation, lag  1     0.8250475036301713
Correlation, lag  2     0.4820939752221447
Correlation, lag  3     0.0661838628463355
Correlation, lag  4     -0.29310582259154694
Correlation, lag  5     -0.5386625150984725
Correlation, lag  6     -0.6496282831082484
Correlation, lag  7     -0.549838477475518
Correlation, lag  8     -0.3185413831731613
Correlation, lag  9     0.02011606382325631
Correlation, lag  10     0.41176952584284515
Correlation, lag  11     0.7322731003955263
Correlation, lag  12     0.8944851710984221
Correlation, lag  13     0.7299213692607464
Correlation, lag  14     0.4055214152186815
Correlation, lag  15     0.0031237974667481893
Correlation, lag  16     -0.34159925855108353
Correlation, lag  17     -0.5745510163732713
Correlation, lag  18     -0.681216824190257
Correlation, lag  19     -0.5902172854543253
Correlation, lag  20     -0.37531995989940625
Correlation, lag  21     -0.04750388422754723
Correlation, lag  22     0.3369494305895757
Correlation, lag  23   

In [89]:
cycleLen = period   
avg = []            
numCycles = int(len(dfTC)/cycleLen + 0.5)
for j in range(cycleLen):
    if j + (numCycles-1) * cycleLen < len(dfTC):
        d = dfTC.iloc[range(j,j + (numCycles-1) * cycleLen+1,cycleLen)]['R1Private']
        avg.append(sum(d)/len(d))
    else:
        d = dfTC.iloc[range(j,j + (numCycles-2) * cycleLen+1,cycleLen)]['R1Private']
        avg.append(sum(d)/len(d))
dfTC['C'] = np.NaN
for i in range(len(dfTC)):
    dfTC.loc[dfTC.index[i], 'C'] = avg[i % cycleLen]
    


In [90]:
dfTC['R2Private'] = dfTC['R1Private'] - dfTC['C']
dfTC['R2ErrorPrivate'] = abs(dfTC['R2Private']/dfTC['PrivateConstruction'])
dfTC['fit'] = dfTC['MovAvg'] + dfTC['C']
print(dfTC)
print('Average Error: ', sum(dfTC['R2ErrorPrivate'])/len(dfTC))


            Month MonthYear  TotalConstruction  PrivateConstruction  \
2003-07-31     19     3-Jul              82971                62216   
2003-08-31     20     3-Aug              85191                63234   
2003-09-30     21     3-Sep              83841                61767   
2003-10-31     22     3-Oct              83133                62813   
2003-11-30     23     3-Nov              77915                60115   
...           ...       ...                ...                  ...   
2012-04-30    124    12-Apr              66223                44964   
2012-05-31    125    12-May              72240                48342   
2012-06-30    126    12-Jun              77876                50900   
2012-07-31    127    12-Jul              78897                51794   
2012-08-31    128    12-Aug              81712                53073   

            PublicConstruction        MovAvg         sqErr    R1Private  \
2003-07-31               20755  57969.430556  1.803335e+07  4246.569444 

Repeat using Public Construction


In [102]:
slope,intercept,r_value,p_value,std_err = stats.linregress(dfTC.index,dfTC['PublicConstruction'])
print('intercept =',intercept,' slope =',slope,'p_value =',p_value)

intercept = 18327.939893765724  slope = 52.00857586303297 p_value = 3.028349007007645e-09


Private Construction is also not linear P-value = 3.028

In [104]:
def sqErr(row):
    return (row['PublicConstruction'] - row['MovAvg'])**2

dfTC['MovAvg'] = np.NaN
dfTC['sqErr'] = np.NaN

dfTC.index = pd.DatetimeIndex(freq='m', start=pd.Timestamp(year=2002, month=1, day=31), periods = len(dfTC['PublicConstruction']))

window = 36 
window = window - window % 2

for i in range (int(window/2),dfTC.shape[0]-int(window/2)):
    dfTC.loc[dfTC.index[i],'MovAvg'] = (0.5*dfTC.iloc[i - int(window/2)]['PublicConstruction'] + dfTC.iloc[i - int(window/2)+1:i + int(window/2)]['PublicConstruction'].sum() + .5*dfTC.iloc[i + int(window/2)]['PublicConstruction'])/float(window)

dfTC['sqErr'] = (dfTC['PublicConstruction'] - dfTC['MovAvg'])**2

dfTC.dropna(how='any',inplace=True)

C:\Users\austi\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Creating a DatetimeIndex by passing range endpoints is deprecated.  Use `pandas.date_range` instead.
  import sys


In [105]:
dfTC['R1Public'] = dfTC['PublicConstruction'] - dfTC['MovAvg']
dfTC['R1ErrorPublic'] = abs((dfTC['PublicConstruction'] - dfTC['R1Public'])/dfTC['PublicConstruction'])
dfTC.style.format({
    'MovAvg': '{:.1f}'.format,
    'sqErr': '{:,.1f}'.format,
    'R1Public': '{:,.1f}'.format,
    'R1ErrorPublic': '{:,.3f}'.format
})

,Month,MonthYear,TotalConstruction,PrivateConstruction,PublicConstruction,MovAvg,sqErr,R1Public,R1ErrorPublic
2003-07-31 00:00:00,37,5-Jan,71474,57094,14380,19164.8,"22,894,364.2","-4,784.8",1.333
2003-08-31 00:00:00,38,5-Feb,72048,57531,14517,19273.1,"22,620,328.7","-4,756.1",1.328
2003-09-30 00:00:00,39,5-Mar,81345,65425,15920,19365.8,"11,873,575.9","-3,445.8",1.216
2003-10-31 00:00:00,40,5-Apr,85485,67829,17656,19447.9,"3,210,915.6","-1,791.9",1.101
2003-11-30 00:00:00,41,5-May,92959,72862,20097,19547.1,"302,423.6",549.9,0.973
2003-12-31 00:00:00,42,5-Jun,99632,77258,22375,19659.2,"7,375,448.9","2,715.8",0.879
2004-01-31 00:00:00,43,5-Jul,103158,80480,22679,19774.3,"8,437,411.2","2,904.7",0.872
2004-02-29 00:00:00,44,5-Aug,106706,82629,24078,19890.1,"17,538,762.3","4,187.9",0.826
2004-03-31 00:00:00,45,5-Sep,103269,79584,23685,20002.7,"13,559,169.6","3,682.3",0.845
2004-04-30 00:00:00,46,5-Oct,102339,80820,21519,20125.3,"1,942,422.9","1,393.7",0.935


In [106]:
maxCorr = 0.0
period = np.NaN
for i in range(1,37):
    corr = dfTC['R1Public'].autocorr(lag=i)
    print('Correlation, lag ',i,'   ',corr)
    if corr > maxCorr:
        maxCorr = corr
        period = i
print('period = ',period,'     Maximum Correlation = ',maxCorr)

Correlation, lag  1     0.8542129071415239
Correlation, lag  2     0.483778620982585
Correlation, lag  3     -0.014002385567195744
Correlation, lag  4     -0.49819401930502843
Correlation, lag  5     -0.829508118249748
Correlation, lag  6     -0.9523747067186772
Correlation, lag  7     -0.8405728300831049
Correlation, lag  8     -0.5008033539105848
Correlation, lag  9     -0.003965801651178323
Correlation, lag  10     0.4813005165722554
Correlation, lag  11     0.8423140416589874
Correlation, lag  12     0.9831515976360988
Correlation, lag  13     0.8538494269790771
Correlation, lag  14     0.4926720370364964
Correlation, lag  15     -0.00853743257335958
Correlation, lag  16     -0.49556326137663687
Correlation, lag  17     -0.8297630324907732
Correlation, lag  18     -0.9531763107082359
Correlation, lag  19     -0.8427618336291858
Correlation, lag  20     -0.5016993047185025
Correlation, lag  21     -0.004620756735558917
Correlation, lag  22     0.48718348876780393
Correlation, lag  2

In [108]:
cycleLen = period   
avg = []            
numCycles = int(len(dfTC)/cycleLen + 0.5)
for j in range(cycleLen):
    if j + (numCycles-1) * cycleLen < len(dfTC):
        d = dfTC.iloc[range(j,j + (numCycles-1) * cycleLen+1,cycleLen)]['R1Public']
        avg.append(sum(d)/len(d))
    else:
        d = dfTC.iloc[range(j,j + (numCycles-2) * cycleLen+1,cycleLen)]['R1Public']
        avg.append(sum(d)/len(d))
dfTC['C'] = np.NaN
for i in range(len(dfTC)):
    dfTC.loc[dfTC.index[i], 'C'] = avg[i % cycleLen]

In [109]:
dfTC['R2Public'] = dfTC['R1Public'] - dfTC['C']
dfTC['R2ErrorPublic'] = abs(dfTC['R2Public']/dfTC['PublicConstruction'])
dfTC['fit'] = dfTC['MovAvg'] + dfTC['C']
print(dfTC)
print('Average Error: ', sum(dfTC['R2ErrorPublic'])/len(dfTC))

            Month MonthYear  TotalConstruction  PrivateConstruction  \
2003-07-31     37     5-Jan              71474                57094   
2003-08-31     38     5-Feb              72048                57531   
2003-09-30     39     5-Mar              81345                65425   
2003-10-31     40     5-Apr              85485                67829   
2003-11-30     41     5-May              92959                72862   
...           ...       ...                ...                  ...   
2009-04-30    106    10-Oct              73470                45539   
2009-05-31    107    10-Nov              68019                42805   
2009-06-30    108    10-Dec              60202                38928   
2009-07-31    109    11-Jan              50955                32377   
2009-08-31    110    11-Feb              50544                32069   

            PublicConstruction        MovAvg         sqErr     R1Public  \
2003-07-31               14380  19164.805556  2.289436e+07 -4784.805556 